This notebook is used in the improvement of our models.
The test do variation on the confidence interval of AR

In [ ]:
import os
import pandas as pd
import numpy as np
import time
from datetime import timedelta
from pmdarima.arima import ARIMA
from pmdarima.arima import auto_arima
from statsmodels.tsa.ar_model import AR
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [ ]:
path = "dataset/test"
files = ["2019-05-13 00-00-01 scd30.csv","2019-05-28 01-00-01 scd30.csv","2019-06-13 01-00-02 scd30.csv",
          "2019-05-20 00-00-01 scd30.csv","2019-06-10 01-00-02 scd30.csv","2019-05-29 01-00-02 scd30.csv"]

In [ ]:
def custom_resampler(array_like):
    return array_like[0]

In [ ]:
import numpy as np
import time
from statsmodels.tsa.ar_model import AR
from datetime import timedelta

class Test_AR():

    def __init__(self,bufferSize,nbPred,nbPreMin,isLog,confint=30):
        self.avgAR = 0
        self.bufferSize = bufferSize
        self.nbPred = nbPred
        self.isLog = isLog
        self.nbPreMin = nbPreMin
        self.confint = confint
    
    def pred(self,ts,valid,f):
        start = time.time()
        modelAR = AR(ts).fit()
        try:
            pred = modelAR.predict(ts.index[-1],ts.index[-1] + timedelta(minutes=self.nbPred))
            end = time.time()
            f.write(str(end-start)+",")
            inPred = 0
            if(self.isLog):
                pred = np.exp(pred)
                ts = np.exp(ts)
            
            for k in range(0,len(pred)):
                if(valid[k]>= pred[k] - self.confint and valid[k]<= pred[k] + self.confint):
                    inPred += 1
            percent = inPred/len(pred)
        except Exception as e:
            #print(e)
            inPred = 0
            percent = 0
        f.write(str(percent))
        self.avgAR += percent

    def getAvg(self):
        return self.avgAR

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d %H-%M-%S')

for i in range(0,len(files)):
    
    data = pd.read_csv(path+'/'+files[i], parse_dates=['Time'], index_col='Time',date_parser=dateparse)
    idx = pd.date_range(start=data.index[0],end=data.index[-1],freq="S")
    dataFix = data.reindex(idx, fill_value=np.nan)
    dataFix = dataFix.interpolate()
    date_rng = pd.date_range(start=dataFix.index[0], end=dataFix.index[-1], freq='5T')
    
    j = len(date_rng) - 8
    nbPredPMin = 12
    
    confintArray = [5,10,15,20,25,30,35,40]
    
    dataSampling = ["5S"]
    sampling = "5S"
    bufsize = [4]
    
    f = open("results/confint/results-confint.csv","a")
    f.write(files[i]+"\n")
    f.write("Iteration: "+str(i)+"\n")
    print(files[i]+"\n")
    print("Iteration: "+str(i))
    f.close()
    z = 0
    for confint in confintArray:
        
        print(confint)
        for l in range(0,len(bufsize)):
            ARLog = Test_AR(20,15,nbPredPMin,True,confint)
            kl = 0
            for k in range(0,j):
                f = open("results/confint/results-confint"+str(i)+"-"+str(z)+".csv","a")
                f.write(str(k)+",")

                ###########
                if(True):
                    try:
                        date_rng = pd.date_range(start=dataFix.index[0], end=dataFix.index[-1], freq="5T")
                        tsFix = dataFix[date_rng[k]:date_rng[k+bufsize[l]]]["#CO2"]
                        valid = dataFix[date_rng[k+bufsize[l]]:date_rng[k+bufsize[l]+3]]["#CO2"]

                        tsFix = tsFix.resample(sampling).mean()
                        ts_log = np.log(tsFix)  
                        valid = valid.resample(sampling).apply(custom_resampler)
                        ARLog.pred(ts_log,valid,f)
                    except:
                        continue

                f.write("\n")
                f.close()
                kl +=1
            f = open("results/confint/results-confint.csv","a")
            f.write("AVG AR "+str(bufsize[l])+" :"+str(ARLog.getAvg()/kl)+"\n")
            print("AVG AR "+str(bufsize[l])+" :"+str(ARLog.getAvg()/kl))
            f.close()
        z+=1

    print("\n")